In [1]:
from datetime import datetime as dt  #to manipulate dates

import numpy as np  #to cleanse data
import openpyxl
import pandas as pd
import plotly.express as px  #to create interactive charts
import plotly.graph_objects as go  #to create interactive charts
from datetime import datetime

In [2]:
ytd_data = pd.read_csv(
    'D:\\Sean\\Documents\\Personal\\Budget\\YTD_downloads\\2025\\wellsfargo.csv', header=None, names=['Date','Amount', 'delete1', 'delete2', 'Description'], parse_dates=['Date']
)

del ytd_data['delete1']
del ytd_data['delete2']

ytd_data['Account'] = 'wells_fargo_checking'
ytd_data.rename(columns={'Date':'Trans. Date'}, inplace=True)
ytd_data['Date'] = pd.to_datetime(ytd_data['Trans. Date'].dt.strftime('%m/%Y'), format='%m/%Y')


In [3]:
ytd_data.tail(10)

,Trans. Date,Amount,Description,Account,Date
103,2025-01-15,1000.00,ZELLE FROM SHARON LEWIS ON 01/15 REF # 3BW01XV...,wells_fargo_checking,2025-01-01
104,2025-01-14,-230.00,ZELLE TO LEWIS SHARON ON 01/14 REF #PP0YFKTZK7...,wells_fargo_checking,2025-01-01
105,2025-01-13,-150.00,MONEY TRANSFER AUTHORIZED ON 01/12 CASH APP*CA...,wells_fargo_checking,2025-01-01
106,2025-01-13,-22.21,PURCHASE AUTHORIZED ON 01/11 TRADER JOE S #06 ...,wells_fargo_checking,2025-01-01
107,2025-01-13,-20.00,MONEY TRANSFER AUTHORIZED ON 01/11 CASH APP*CA...,wells_fargo_checking,2025-01-01
108,2025-01-09,-100.00,ZELLE TO LEWIS JR DON ON 01/09 REF #PP0YF4Y6GD...,wells_fargo_checking,2025-01-01
109,2025-01-09,-8.25,PURCHASE AUTHORIZED ON 01/07 FCCC SMOG REF SIT...,wells_fargo_checking,2025-01-01
110,2025-01-09,370.89,Nationwide PAYROLLAB3 HRP0010953231 Sean Lewis,wells_fargo_checking,2025-01-01
111,2025-01-06,-217.00,ZELLE TO LEWIS JR DON ON 01/06 REF #PP0YDVM2K4...,wells_fargo_checking,2025-01-01
112,2025-01-02,-750.00,ZELLE TO LEWIS JR DON ON 01/02 REF #PP0YDK8MGJ...,wells_fargo_checking,2025-01-01


In [16]:
# Discover Transactions
mapping = {
    # Direct Deposit
    'income': ['PAYROLL'],
    'family': ['JAYELIN', 'JUSTINA', 'SHARON', 'DORRY'],
    'purchases': ['THERAPY STORES', 'AMAZON', 'NOISEP'],
    'other_people': ['NIKA'],
    'venmo': ['VENMO', 'JONATHAN'],
    'refunds': ['PREARRANGE', 'STUBHUB CONS PAYMENTS 240117 000000017167884 9016004299-SEAN LEWIS'],
    'cell_phone': ['T-Mobile', 'TMOBILE'],
    'la_fitness': ['L A FITNESS'],
    'transfer': ['TRANSFER','MOBILE DEPOSIT : REF NUMBER :314230152812'],
    'student_loan': ['DEPT EDUCATION'],
    'dentist': ['OANH LE'],
    'care_credit': ['Synchrony Bank CC'],
    'fees': ['FEE'],
    'gas': ['SHELL', 'CHEVRON', 'ARCO', '76', 'A&A','PURCHASE AUTHORIZED ON 12/21 PALO ALTO GAS & PALO ALTO CA P000000885077547 CARD 1081','BELMONT ALLIANCE'],
    'laundry': ['Cash eWithdrawal in Branch'],
    'parking': ['REDWOOD CITY PARKI REDWOOD CITY', 'STADIUM GARAGE','PURCHASE AUTHORIZED ON 10/25 STADIUM GARAGE BERKELEY CA S584300071995368 CARD 1081'],
    'ATM': ['ATM'],
    'Vincent': ['Kai Feng'],
    'GT_China': ['Candice Mclemore', 'Alipay', 'CHN', 'Macau', 'HKG'],
    'Australia': ['SYDNEY AUS', 'AUS'],
    'Mexico': ['CIUDAD', 'MEX', 'GetYourGuide'],
    'learning': ['BRITISH SWIM SCHOO'],
    'parking': [r'LAZ P.*', r'Park.*'],
    'Don': ['LEWIS JR', 'DON'],
    'Shevon':['Shevon'],
    'haircut': ['DICE'],
    'Cris' : ['Chris'],
    'general_festival': ['Stockton 9'],
    'misc': ['Lewis Sean'],
    'restaurants': ['West Town Bank', 'PIZZA HUT'],
    'movies': ['AMC'],
    'groceries': ['WEIS', 'BJS'],
    'monitor': ['Best Buy'],
    'sport_stuff': ['BAUER*'],
    'India': ['PURCHASE AUTHORIZED ON 07/29 MinistryofExternal Mumbai IND'],
    'movies': ['CINEPOLIS'],
    'groceries': ['TRADER JOE'],
    'parking': ['METER MTA', 'PAYBYPHONE','CITY OF BURLINGAME'],
    'gym': ['24 HOUR FITNESS']

}
for k, v in mapping.items():
    ytd_data.loc[ytd_data.Description.str.contains('|'.join(v), case=False), 'Short_Desc'] = k


In [17]:
ytd_data[ytd_data['Short_Desc'].isnull()].sort_values(by='Amount', ascending=False)

,Trans. Date,Amount,Description,Account,Date,Short_Desc


In [17]:
ytd_data.loc[ytd_data['Short_Desc'].isnull(), 'Short_Desc'] = 'unsure'

In [22]:
reocurring_list = ['cell_phone', 'la_fitness', 'student_loan','care_credit']
ytd_data.loc[ytd_data.Short_Desc.isin(reocurring_list), 'Reoccurring_Flag'] = 'Yes'
ytd_data.loc[~ytd_data.Short_Desc.isin(reocurring_list), 'Reoccurring_Flag'] = 'No'

mapping = {
    'Housing': [],
    'Transportation': ['parking'],
    'Food': ['groceries'],
    'Insurance': [],
    'Utilities': ['cell_phone','gas','laundry'],
    'Medical': ['medical','care_credit','dentist'],
    'Government': ['student_loan'],
    'Savings': [],
    'Personal_Spending': ['la_fitness','ATM','restaurants','learning', 'purchases','haircut','gym','monitor'],
    'Recreation_Entertainment': ['general_festival', 'movies','sport_stuff'],
    'Travel': ['GT_China','Australia','Mexico','air_travel', 'airport','India'],
    'Misc': ['unsure','misc'],
    'People': ['family','venmo','other_people','transfer','Don','Vincent','Shevon','Cris'],
    'Payment_and_Interest': ['fees'],
    
    'Income':['income','refunds']
}

for k, v in mapping.items():
    ytd_data.loc[ytd_data.Short_Desc.isin(v), 'Category_2'] = k

In [23]:
ytd_data[ytd_data['Category_2'].isnull()].sort_values(by='Description', ascending=False).sort_values(by='Short_Desc',
                                                                                                     ascending=False)

,Trans. Date,Amount,Description,Account,Date,Short_Desc,Reoccurring_Flag,Category_2


In [24]:
x = ytd_data.groupby(['Reoccurring_Flag']).size().reset_index(name='count')

x.sort_values(by='count', ascending=False)

,Reoccurring_Flag,count
0,No,109
1,Yes,4


In [25]:
ytd_data.reset_index(inplace=True)
ytd_sorted = ytd_data.sort_values(by=['Trans. Date', 'index'])

# Define the new row with datetime objects for date columns
new_row = pd.DataFrame({
    'index': [0],  # Ensure all entries are in list format to align with DataFrame structure
    'Trans. Date': [datetime.strptime('1/1/2023', '%m/%d/%Y')],  # Matching format and conversion
    'Amount': [616.53],
    'Description': ['Balance'],
    'Account': ['wells_fargo_checking'],
    'Date': [pd.to_datetime('1/1/2023')],  # This is a string representation of the date, so it's okay
    'Short_Desc': ['Balance'],
    'Reoccurring_Flag': ['No'],
    'Category_2': ['Balance'],

}, index=[0])

# Concatenate the new_row DataFrame with the existing ytd_sorted DataFrame
ytd_sorted = pd.concat([new_row, ytd_sorted], ignore_index=True)

# Ensure 'Sort_Date' is also in datetime format for the entire DataFrame
ytd_sorted['Trans. Date'] = pd.to_datetime(ytd_sorted['Trans. Date'])
ytd_sorted

,index,Trans. Date,Amount,Description,Account,Date,Short_Desc,Reoccurring_Flag,Category_2
0,0,2023-01-01,616.53,Balance,wells_fargo_checking,2023-01-01,Balance,No,Balance
1,112,2025-01-02,-750.00,ZELLE TO LEWIS JR DON ON 01/02 REF #PP0YDK8MGJ...,wells_fargo_checking,2025-01-01,Don,No,People
2,111,2025-01-06,-217.00,ZELLE TO LEWIS JR DON ON 01/06 REF #PP0YDVM2K4...,wells_fargo_checking,2025-01-01,Don,No,People
3,108,2025-01-09,-100.00,ZELLE TO LEWIS JR DON ON 01/09 REF #PP0YF4Y6GD...,wells_fargo_checking,2025-01-01,Don,No,People
4,109,2025-01-09,-8.25,PURCHASE AUTHORIZED ON 01/07 FCCC SMOG REF SIT...,wells_fargo_checking,2025-01-01,gas,No,Utilities
...,...,...,...,...,...,...,...,...,...
109,4,2025-10-20,-1000.00,ZELLE TO LEWIS SHARON ON 10/18 REF # WFCT0ZDFN...,wells_fargo_checking,2025-10-01,family,No,People
110,5,2025-10-20,1000.00,SoFi Bank TRANSFER SD1300 SLewis Sean Lewis:67...,wells_fargo_checking,2025-10-01,misc,No,Misc
111,0,2025-10-21,-44.07,PURCHASE AUTHORIZED ON 10/20 A&A GAS BURLINGAM...,wells_fargo_checking,2025-10-01,gas,No,Utilities
112,1,2025-10-21,80.00,ZELLE FROM JONATHAN W ABADA ON 10/21 REF # JPM...,wells_fargo_checking,2025-10-01,venmo,No,People


In [26]:
ytd_sorted['CumAmount'] = ytd_sorted['Amount'].cumsum()
ytd_sorted['Year'] = ytd_sorted['Trans. Date'].dt.year

ytd_sorted

,index,Trans. Date,Amount,Description,Account,Date,Short_Desc,Reoccurring_Flag,Category_2,CumAmount,Year
0,0,2023-01-01,616.53,Balance,wells_fargo_checking,2023-01-01,Balance,No,Balance,616.53,2023
1,112,2025-01-02,-750.00,ZELLE TO LEWIS JR DON ON 01/02 REF #PP0YDK8MGJ...,wells_fargo_checking,2025-01-01,Don,No,People,-133.47,2025
2,111,2025-01-06,-217.00,ZELLE TO LEWIS JR DON ON 01/06 REF #PP0YDVM2K4...,wells_fargo_checking,2025-01-01,Don,No,People,-350.47,2025
3,108,2025-01-09,-100.00,ZELLE TO LEWIS JR DON ON 01/09 REF #PP0YF4Y6GD...,wells_fargo_checking,2025-01-01,Don,No,People,-450.47,2025
4,109,2025-01-09,-8.25,PURCHASE AUTHORIZED ON 01/07 FCCC SMOG REF SIT...,wells_fargo_checking,2025-01-01,gas,No,Utilities,-458.72,2025
...,...,...,...,...,...,...,...,...,...,...,...
109,4,2025-10-20,-1000.00,ZELLE TO LEWIS SHARON ON 10/18 REF # WFCT0ZDFN...,wells_fargo_checking,2025-10-01,family,No,People,-811.16,2025
110,5,2025-10-20,1000.00,SoFi Bank TRANSFER SD1300 SLewis Sean Lewis:67...,wells_fargo_checking,2025-10-01,misc,No,Misc,188.84,2025
111,0,2025-10-21,-44.07,PURCHASE AUTHORIZED ON 10/20 A&A GAS BURLINGAM...,wells_fargo_checking,2025-10-01,gas,No,Utilities,144.77,2025
112,1,2025-10-21,80.00,ZELLE FROM JONATHAN W ABADA ON 10/21 REF # JPM...,wells_fargo_checking,2025-10-01,venmo,No,People,224.77,2025


In [27]:
ytd_sorted['Counter'] = ytd_sorted.groupby('Short_Desc').cumcount() + 1
ytd_sorted

,index,Trans. Date,Amount,Description,Account,Date,Short_Desc,Reoccurring_Flag,Category_2,CumAmount,Year,Counter
0,0,2023-01-01,616.53,Balance,wells_fargo_checking,2023-01-01,Balance,No,Balance,616.53,2023,1
1,112,2025-01-02,-750.00,ZELLE TO LEWIS JR DON ON 01/02 REF #PP0YDK8MGJ...,wells_fargo_checking,2025-01-01,Don,No,People,-133.47,2025,1
2,111,2025-01-06,-217.00,ZELLE TO LEWIS JR DON ON 01/06 REF #PP0YDVM2K4...,wells_fargo_checking,2025-01-01,Don,No,People,-350.47,2025,2
3,108,2025-01-09,-100.00,ZELLE TO LEWIS JR DON ON 01/09 REF #PP0YF4Y6GD...,wells_fargo_checking,2025-01-01,Don,No,People,-450.47,2025,3
4,109,2025-01-09,-8.25,PURCHASE AUTHORIZED ON 01/07 FCCC SMOG REF SIT...,wells_fargo_checking,2025-01-01,gas,No,Utilities,-458.72,2025,1
...,...,...,...,...,...,...,...,...,...,...,...,...
109,4,2025-10-20,-1000.00,ZELLE TO LEWIS SHARON ON 10/18 REF # WFCT0ZDFN...,wells_fargo_checking,2025-10-01,family,No,People,-811.16,2025,6
110,5,2025-10-20,1000.00,SoFi Bank TRANSFER SD1300 SLewis Sean Lewis:67...,wells_fargo_checking,2025-10-01,misc,No,Misc,188.84,2025,6
111,0,2025-10-21,-44.07,PURCHASE AUTHORIZED ON 10/20 A&A GAS BURLINGAM...,wells_fargo_checking,2025-10-01,gas,No,Utilities,144.77,2025,20
112,1,2025-10-21,80.00,ZELLE FROM JONATHAN W ABADA ON 10/21 REF # JPM...,wells_fargo_checking,2025-10-01,venmo,No,People,224.77,2025,3


In [28]:
columns = ['Trans. Date', 'Description', 'Amount', 'Category', 'Short_Desc', 'Reoccurring_Flag',
                     'Category_2', 'Date', 'Year', 'Sort_Date', 'CumAmount', 'Counter', 'Account']
ytd_sorted['Category'] = 'NA'
ytd_sorted['Sort_Date'] = ytd_sorted['Trans. Date']

wf = ytd_sorted[columns]

In [29]:
wf.to_excel('D:\\Sean\\Documents\\Personal\\Budget\\2025_output\\WF_Curated_Bills.xlsx')